## Read raw data source

In [1]:
import requests 

#### Fetch Wiki page

In [2]:
canada_postal_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
canada_postal_raw = requests.get(canada_postal_url).text

#### Define function for process each table row

In [3]:
def process_row(raw_table_row):
    """
    Function for convert row to the df line
    """
    raw_table_row = raw_table_row.strip()
    if '<a href' in raw_table_row:
        raw_table_row = raw_table_row.split('">')[1].split('</a>')[0]
    return raw_table_row

#### Convert raw text to the list of the rows

Used simple python tools, as we opperate with small amout of the data

In [4]:
postal_table = (
    canada_postal_raw
    .split('<table class="wikitable sortable')[1]
    .split('<tbody>')[1]
    .split('</tbody>')[0]
    .replace('</th>','').replace('</tr>','').replace('</td>','')
)

data = [tuple(map(process_row, row.strip().split('<td>')[1:])) for row in postal_table.split('<tr>')][2:]

#### Convert list of the row to the df and apply some cleaning

In [5]:
import pandas as pd 
index_df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
index_df = index_df[index_df['Borough']!='Not assigned']
index_df.loc[index_df['Neighborhood']=='Not assigned', 'Neighborhood'] = index_df.loc[index_df['Neighborhood']=='Not assigned', 'Borough']
index_df = index_df.groupby(['PostalCode', 'Borough']).agg(lambda x:', '.join(list(x))).reset_index(level=[0,1])

#### Print df head 

In [6]:
index_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Print df shape 

In [7]:
index_df.shape

(103, 3)

## Add location

### Donwload location csv 

In [8]:
file_path = './../data/Geospatial_data.csv'
file_url = 'https://cocl.us/Geospatial_data'
!wget -O {file_path} {file_url}

--2019-03-11 19:47:53--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-11 19:47:57--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-11 19:47:58--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-03-11 19

### Read downloaded csv into pandas dataframe

In [9]:
zip_df = pd.read_csv(file_path)

In [10]:
zip_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Join dfs to obrain zips location

In [11]:
location_df = (
    pd.
    concat([zip_df.set_index('Postal Code'),index_df.set_index('PostalCode')], axis=1, join='inner').
    reset_index().
    rename(columns={"index": "PostalCode"})
)

In [12]:
location_df.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [13]:
location_df.shape

(103, 5)

In [14]:
import folium

In [15]:
def show_map(location_df):
    """
    Function for create map
    """
    latitude = location_df['Latitude'].mean()
    longitude = location_df['Longitude'].mean()
    torronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

    # add markers to map
    for lat, lng, borough, neighborhood in zip(location_df['Latitude'], location_df['Longitude'], location_df['Borough'], location_df['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(torronto_map)  

    return torronto_map

### Display map of all location

In [16]:
show_map(location_df)

### Filter location, which contains Torronto 

In [17]:
torronto_location_df = location_df[location_df['Borough'].str.contains("Toronto")]

In [18]:
torronto_location_df.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
37,M4E,43.676357,-79.293031,East Toronto,The Beaches
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale"
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar"
43,M4M,43.659526,-79.340923,East Toronto,Studio District
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park


In [19]:
show_map(torronto_location_df)

## Load information from forsquare 

### Setup credentionals and request arguments

In [20]:
LIMIT = 100
radius = 500

In [21]:
CLIENT_ID = '****' 
CLIENT_SECRET = '*****' 
VERSION = '20180605' # Foursquare API version

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
torronto_venues = getNearbyVenues(names=torronto_location_df['Neighborhood'],
                                   latitudes=torronto_location_df['Latitude'],
                                   longitudes=torronto_location_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

### Load venu of each lication

In [29]:
print(torronto_venues.shape)
torronto_venues.head()

(1689, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


## Get some insights 

In [30]:
torronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,81,81,81,81,81,81
"Chinatown, Grange Park, Kensington Market",97,97,97,97,97,97
Christie,16,16,16,16,16,16


## Transform data to define 10 features, which define most popular place

In [31]:
# one hot encoding
torronto_onehot = pd.get_dummies(torronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torronto_onehot['Neighborhood'] = torronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torronto_onehot.columns[-1]] + list(torronto_onehot.columns[:-1])
torronto_onehot = torronto_onehot[fixed_columns]

torronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
torronto_onehot.shape

(1689, 236)

In [33]:
torronto_grouped = torronto_onehot.groupby('Neighborhood').mean().reset_index()
torronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00


In [34]:
torronto_grouped.shape

(38, 236)

In [47]:
def return_most_common_venues(row, num_top_venues=5):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
import numpy as np

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Bar,Gym,Asian Restaurant,Burger Joint,American Restaurant,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Italian Restaurant,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Restaurant,Steakhouse,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Café,Performing Arts Venue,Breakfast Spot,Coffee Shop,Bar,Restaurant,Stadium,Caribbean Restaurant,Italian Restaurant,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden,Skate Park,Park,Farmers Market,Fast Food Restaurant,Brewery,Burrito Place,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Boutique,Airport Gate,Airport


## Apply Clustering

In [50]:
from sklearn.cluster import KMeans

In [51]:
# set number of clusters
kclusters = 5

torronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 3, 0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [52]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
torronto_merged = torronto_location_df
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
torronto_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar",0,Park,Brewery,Sandwich Place,Italian Restaurant,Sushi Restaurant,Intersection,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater
43,M4M,43.659526,-79.340923,East Toronto,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Yoga Studio,Fish Market,Juice Bar,Bookstore
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,2,Park,Swim School,Bus Line,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## Draw clusters on the map

In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [54]:
latitude = torronto_merged['Latitude'].mean()
longitude = torronto_merged['Longitude'].mean()

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Latitude'], torronto_merged['Longitude'], torronto_merged['Neighborhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
def filter_df_by_cluster(df, cluster):
    return df[df['Cluster Labels'] == cluster]

# Result Cluster #0

In [45]:
filter_df_by_cluster(torronto_merged, 0)

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,43.676357,-79.293031,East Toronto,The Beaches,0,Health Food Store,Coffee Shop,Pub,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,M4K,43.679557,-79.352188,East Toronto,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant,Caribbean Restaurant
42,M4L,43.668999,-79.315572,East Toronto,"The Beaches West, India Bazaar",0,Park,Brewery,Sandwich Place,Italian Restaurant,Sushi Restaurant,Intersection,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater
43,M4M,43.659526,-79.340923,East Toronto,Studio District,0,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Yoga Studio,Fish Market,Juice Bar,Bookstore
44,M4N,43.728020,-79.388790,Central Toronto,Lawrence Park,0,Park,Swim School,Bus Line,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
45,M4P,43.712751,-79.390197,Central Toronto,Davisville North,0,Hotel,Breakfast Spot,Burger Joint,Food & Drink Shop,Dance Studio,Clothing Store,Sandwich Place,Park,Gym,Electronics Store
46,M4R,43.715383,-79.405678,Central Toronto,North Toronto West,0,Coffee Shop,Clothing Store,Sporting Goods Shop,Bagel Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
47,M4S,43.704324,-79.388790,Central Toronto,Davisville,0,Sandwich Place,Pizza Place,Dessert Shop,Restaurant,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Pharmacy,Gourmet Shop
48,M4T,43.689574,-79.383160,Central Toronto,"Moore Park, Summerhill East",0,Playground,Gym,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
49,M4V,43.686412,-79.400049,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Pub,American Restaurant,Supermarket,Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,Sushi Restaurant,Convenience Store


# Result Cluster #1

In [46]:
filter_df_by_cluster(torronto_merged, 1)

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,M5N,43.711695,-79.416936,Central Toronto,Roselawn,1,Garden,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


# Result Cluster #2


In [56]:
filter_df_by_cluster(torronto_merged, 2)

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,43.72802,-79.38879,Central Toronto,Lawrence Park,2,Park,Swim School,Bus Line,Women's Store,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


# Result Cluster #3

In [58]:
filter_df_by_cluster(torronto_merged, 3)

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,M4T,43.689574,-79.38316,Central Toronto,"Moore Park, Summerhill East",3,Playground,Gym,Tennis Court,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


# Result Cluster #4

In [59]:
filter_df_by_cluster(torronto_merged, 4)

,PostalCode,Latitude,Longitude,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,M4W,43.679563,-79.377529,Downtown Toronto,Rosedale,4,Park,Playground,Trail,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
